This notebook demonstrates A star pathfinding on a pointcloud using Delaunay triangulation to connect the cloud and form a graph.  
A* implementation from [Red Blob Games](https://www.redblobgames.com/pathfinding/a-star/implementation.html)

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import laspy
import scipy.spatial as spatial
# %matplotlib widget

In [2]:
# https://stackoverflow.com/a/23700182
def find_neighbors(pindex, triang):
    return triang.vertex_neighbor_vertices[1][triang.vertex_neighbor_vertices[0][pindex]:triang.vertex_neighbor_vertices[0][pindex+1]]

In [3]:
inFile = laspy.file.File("points.las", mode = "r")

In [37]:
plt.close("all")
points = np.array([inFile.x,inFile.y,inFile.z]).T[:100]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0],points[:,2],points[:,2])
plt.show()

FigureCanvasNbAgg()

In [34]:
# Options: http://www.qhull.org/html/qh-optq.htm
tri = spatial.Delaunay(points, qhull_options = "Qbb Qc Qz Qx QbB")

The code below plots the connections from 'pidx' to its neighbours. Try changing pidx to see which points are connected.

In [38]:
pidx = 25
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tri.points[:,0],tri.points[:,1],tri.points[:,2])
ax.scatter(tri.points[pidx,0],tri.points[pidx,1],tri.points[pidx,2],'r',s=100,alpha=1)
for p in tri.points[find_neighbors(pidx, tri)]:
    plt.plot([p[0],tri.points[pidx,0]],[p[1],tri.points[pidx,1]],[p[2],tri.points[pidx,2]],'k')

FigureCanvasNbAgg()

In [64]:
class PointCloud:
    def __init__(self, tri):
        self.tri = tri
    
    def in_bounds(self, idx):
        return (0 <= idx and idx < self.tri.points.size)
    
    def passable(self, id):
        return True
    
    def neighbors(self, idx):
        # https://stackoverflow.com/a/23700182
        return self.tri.vertex_neighbor_vertices[1][self.tri.vertex_neighbor_vertices[0][idx]:self.tri.vertex_neighbor_vertices[0][idx+1]]

    def cost(self, from_idx, to_idx):
        sq = (self.tri.points[from_idx]-self.tri.points[to_idx])**2
        # print(f"{np.sum(sq)}\t{sq[2]/(sq[0]+sq[1])}")
        #      distance^2   gradient^2
        return np.sum(sq) + 1000*sq[2]/(sq[0]+sq[1])

In [40]:
import heapq
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

In [41]:
def heuristic(a, b):
    return np.sum((a-b)**2)

In [57]:
def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

In [65]:
pc = PointCloud(tri)
start = 25
goal = 5
path_from, cost = a_star_search(pc, start, goal)

path = [goal]
cidx = goal
while path_from[cidx] is not None:
    cidx = path_from[cidx]
    path.append(cidx)
path = np.array(path)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
all_idxs = np.arange(points.shape[0])
not_in_path = np.array([pidx if pidx not in path else -1 for pidx in all_idxs])
not_in_path = not_in_path[np.where(not_in_path != -1)]
ax.scatter(points[not_in_path,0], points[not_in_path,1], points[not_in_path,2], 'k',alpha=1)
ax.scatter(points[path,0],points[path,1],points[path,2],'r',alpha=1)

FigureCanvasNbAgg()